In [2]:
import pandas as pd
import numpy as np


In [3]:
usage_data = pd.read_csv('usage.csv', names=['cust_id', 'event_start', 'event_type', 'rate_plan_id', 'flag_1', 'flag_2', 'duration', 'charge', 'month'])
print(usage_data.shape)
for col in usage_data.columns:
    print(col)
    
# randomly downscale the data for the purpose of analysing it more efficiently.
# the distribution of data and correlations between some attributes remains
# the same as for original dataset, since the dataset is downscaled randomly.
# (the whole dataset should be used for data analysis, however, due to the
# limited computational resources, the fraction of dataset had to be used)
usage_data = usage_data.sample(frac=0.1, replace=True, random_state=1)
print("\nresized:")
print(usage_data.shape)

(22783785, 9)
cust_id
event_start
event_type
rate_plan_id
flag_1
flag_2
duration
charge
month

resized:
(2278378, 9)


In [5]:
# look for the damaged data, such as the elements containing null values.
# as we can see, the data fully contains values, as the missing percentage
# of data for each row is 0%
# if some attributes contain empty values, the billing provider has to be notified ASAP
# for damaged data. the data itself has to be cleansed.
missing_perc = usage_data.isnull().sum()*100/len(usage_data)
print(missing_perc)

cust_id         0.0
event_start     0.0
event_type      0.0
rate_plan_id    0.0
flag_1          0.0
flag_2          0.0
duration        0.0
charge          0.0
month           0.0
dtype: float64


In [6]:
# another thing that is worth observing is that the 'month' attribute in the
# provided dataset contains the same information as the 'event_start' field.
# it's safe to assume that we can drop the 'month' column as we can get
# the same information from 'event_start'.


# exctract month information from 'event_start' field in form YYYY-MM like
# it is defined in 'month' attribute.
event_start_month = usage_data['event_start'].apply(lambda x : x[:7])

# compare whether the values in both lists are equal
print('lists are equal:',(usage_data['month'].tolist() == event_start_month).all())


# drop the 'month' column of the dataframe for further analysis.
# note that this attribute will be removed when loading to the database
usage_data = usage_data.drop(['month'], axis = 1)

lists are equal: True


In [7]:
# continuing looking for data inconsistency and its quality we find that
# the data contains duplications. Thus, we will remove the duplicated rows
# prior to the uploading to the warehouse.
duplicatedRows = usage_data[usage_data.duplicated()]

print('approximately ' +str(round(len(duplicatedRows)/len(usage_data)*100))+'% of data is duplicated')

# drop the duplicated rows
usage_data = usage_data.drop_duplicates()

duplicatedRows = usage_data[usage_data.duplicated()]

print('after deletion: ' +str(round(len(duplicatedRows)/len(usage_data)*100))+'% duplicated data')

approximately 5% of data is duplicated
after deletion: 0% duplicated data


In [8]:
# checkcing the data shape and the unique values count for each attribute.

print(usage_data.shape)

print(usage_data.nunique())

(2166562, 8)
cust_id            9460
event_start     1841464
event_type            4
rate_plan_id       2329
flag_1              207
flag_2                2
duration          10204
charge            82306
dtype: int64


In [20]:
# analyse the existing values for new data insertion. if we know the expected
# range or values for the attributes, we can constrain and reject records that
# exceeds our expectations. also, some errors in data might be found. 
# for example, if we receive a record with 'event_type' value that is different
# from the existing 4 classes, it means our record might be damaged and needs
# to be cleansed.

for col in usage_data.columns:
    if usage_data[col].dtype == np.float64 or usage_data[col].dtype == np.int64:
        print('-',col)
        print('min value:', min(usage_data[col]), ' max value:', max(usage_data[col]), ' mean:', usage_data[col].mean())

print('-', 'event_type')
print('possible values:', usage_data['event_type'].unique())

- cust_id
min value: 1609  max value: 2027509  mean: 1170194.8485886857
- rate_plan_id
min value: 0  max value: 42705  mean: 21679.08809164012
- flag_1
min value: 0  max value: 234  mean: 6.045190029179871
- flag_2
min value: 0  max value: 1  mean: 0.2339125305437832
- duration
min value: 0  max value: 18522  mean: 750.7732356609228
- charge
min value: 0.0  max value: 1496.598432  mean: 0.6891174207661719
- event_type
possible values: ['VOICE' 'DATA' 'SMS' 'MMS']


In [22]:
# after performing this minimal analysis of existing data, we can
# see that it can be expected to receive some duplicated data which
# should be removed prior to loading. also, we might as well avoid using
# the 'month' attribute as 'event_start' contains the same information, thus,
# will allow us save space.

# depending on our expectations, we can constraint the values of new data
# for example, limiting the 'event_type' field to currently existing 4 classes,
# or allowing maximum value of 'flag_1' to be 234 as it currently is. otherwise,
# mark the record as damaged, notify the provider and cleanse the data.

print(usage_data.iloc[0])

cust_id                                827919
event_start     2016-02-12T21:50:27.000+02:00
event_type                              VOICE
rate_plan_id                            16538
flag_1                                      1
flag_2                                      1
duration                                  984
charge                                    0.0
Name: 12710949, dtype: object


# Designing data structure for the Data Warehouse

Once the ETL is performed, the Data Marts will have to be created in the database as was requested by the Product guys. Firstly, after the data validation, cleansing and removal of unnecessary records the data will be uploaded to the warehouse in the initial structure as was provided within the csv file, only without the 'month' attribute. The overall structure of data will be stored in the following form:

|cust_id|event_start|event_type|rate_plan_id|flag_1|flag_2|duration|charge|
|------|------|------|------|------|------|------|------|
|827919  |2016-02-...  |VOICE  | 16538 | 1 | 1 | 984 | 0.0 |
|...  |...  |...  | ... | ... | ... | ... | ... |

As was asked by the Product guys, the distribution of different service types (event_type) and rate plans (rate_plan_id) has to be provided together with the number of customers for each respective field. Thus, two Data Marts were designed to allow for efficient and simple retrieval of such information. 

Data Mart for service types contains 4 rows, each for a different service type, and 3 columns, where first column is a primary key of event_type, second column contains number of existing projects of voice type (usage distribution) and third column - number of distinct customers using such service type.

|event_type|project_count|customers|
|------|------|------|
|VOICE  |distribution of VOICE projects  |#of customers  |
|DATA  |distribution of DATA projects  |#of customers  |
|SMS  |distribution of SMS projects  |#of customers  |
|MMS  |distribution of MMS projects  |#of customers  |

The table for the Rate Plan is designed in the same way, only the distribution and number of customers are counted for each Rate Plan (rate_plan_id as a primary key).

|rate_plan_id|project_count|customers|
|------|------|------|
|0  |distribution of rate plan 0  |#of customers  |
|1  |distribution of rate plan 1  |#of customers  |
|...  |...  |...  